In [1]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import time
import os
import matplotlib.pyplot as plt
import pylab
import pickle
import copy
import processing
import matplotlib.animation as animation
import matplotlib.image as mpimg
import window as w
%matplotlib inline

WORKSPACE_PATH = '/home/takeyama/pywork/ipython/2016-06-27'
RawDataPath=WORKSPACE_PATH+'/raw/'
StudyDataPath=WORKSPACE_PATH+'/study/'
GlaphDataPath=WORKSPACE_PATH+'/glaph/'

In [7]:
from keras.models import Sequential
from keras.datasets import cifar10
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils.visualize_util import model_to_dot, plot
from keras import backend as K
from keras.layers import Input, Dense
from keras.models import Model

In [3]:
dic1=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor1")
dic2=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor2")
dic3=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor3")
dic4=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor4")
dic5=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor5")
dic6=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor6")
dic7=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor7")

In [4]:
dic1['Name']

'MemSensor1'

In [5]:
dic3['Name']

'MemSensor3'

In [6]:
windowSensor1=w.Window()
windowSensor1.SetData('AccX',dic1['AccX'])
wS1=windowSensor1.Compile(windowWidth=16,overlap=0.5)
windowSensor3=w.Window()
windowSensor3.SetData('AccX',dic3['AccX'])
wS3=windowSensor3.Compile(windowWidth=16,overlap=0.5)

AccX is registed now
Build Complete
(254742,)
AccX is registed now
Build Complete
(254742,)


In [8]:
wS3.shape

(15921, 16, 1)

**Sensor1 Learning**

In [ ]:
windoW = wS1
col = windoW.shape[1]
row = windoW.shape[2]
shapeNum=col*row

SaveFileNameEncord='All_encoded'
SaveFileNameDecord='All_decoded'
window_train=Convert3dto2d(windoW)
window_test=Convert3dto2d(windoW)

# this is the size of our encoded representations
encoding_dim = 8  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

shapeNum=window_train.shape[1]

# this is our input placeholder
input_img = Input(shape=(shapeNum,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='tanh')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(shapeNum, activation='linear')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input=input_img, output=decoded)

# this model maps an input to its encoded representation
encoderALL = Model(input=input_img, output=encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoderALL = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='mse')

autoencoder.fit(window_train, window_train,
                nb_epoch=50,
                batch_size=shapeNum/4,
                shuffle=True,
                validation_data=(window_test, window_test))

encoded_imgs = encoderALL.predict(window_test)
decoded_imgs = decoderALL.predict(encoded_imgs)

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameEncord,
                                  encoder.predict(window_test))

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameDecord,
                                  encoder.predict(decoded_imgs))